In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import os
import sys
from ocp_table_tpot.globals import Globals as gd
from tpot import TPOTRegressor
sys.path.insert(0,'..')
from src.models.model import HistoricalMedian,XGBoost,LinearModel,RF,KNN,SVM,mase,TimeSeriesSplitImproved
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,RANSACRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor,ExtraTreesRegressor
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
from keras.initializers import random_uniform
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam,SGD
from keras.initializers import random_uniform,glorot_uniform
from keras.layers import Dense,Dropout,BatchNormalization,Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint,Callback,ReduceLROnPlateau
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler,PolynomialFeatures,StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from skgarden.quantile import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.preprocessing import FunctionTransformer
from copy import copy
from tpot.builtins import StackingEstimator
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline, make_union
from catboost import CatBoostRegressor,Pool,cv
import umap

df_tsfresh = pd.read_pickle(f'../data/processed/train_test_tsfresh_6.pkl').reset_index(level = 0)
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')


In [ ]:
year = 2019
tgt = 'rougher.output.recovery'
X = data_dict[year]['X_train_tsclean']
y = data_dict[year]['y_train_tsclean']
print(f'1) X shape: {X.shape},y: {y.shape}')


X_test=  data_dict[year]['X_test_ts']
print(f'1) Test shape: {X_test.shape}, train: {X.shape}')

inds_y = y[(y[tgt] > 5) & (y[tgt] < 100)].index
inds_common = inds_y
X = X.loc[inds_common,]

print(f'2) Test shape: {X_test.shape}, train: {X.shape}')

y = y.loc[inds_common, tgt]

X = X.sample(frac=0.8,random_state=123).sort_index().dropna()
y= y[X.index]
X_filt = X.filter(regex  ="rougher",axis = 1)
X_filt.shape

# Base models

In [ ]:
n_folds = 8

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.005, random_state=1,max_iter = 2000))

def cv_catboost(model,X_base=None,y=None):
#     pool = Pool(X_base,y)
#     #,task_type='GPU',od_type = 'Iter',od_wait= 15
#     params['task_type'] = "GPU"
#     params['od_type'] = "Iter"
#     params['od_wait'] = 20
#     params['has_time']= True
#     params['plot']=False
#     params['verbose_eval'] = 100
#     params['shuffle'] = False
#     params['fold_count'] = 5
#     params['early_stopping_rounds'] = 15
#     params['random_seed'] = 123
#     scores = cv(pool,params)
   
    cv = KFold(n_folds, shuffle=False, random_state=42)
    scores = []
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)

    for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):
    # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        
        print(y.shape)
        # Do the base
        model.fit(X_train.values,y_train.values.reshape(-1,),eval_set=(X_valid.values, y_valid.values.reshape(-1,)),plot=False,verbose_eval = 300)
        preds  = model.predict(X_valid.values)
        preds_all_base[valid_index] = preds
        true_all[valid_index] = y_valid
        
    oof_scores = mase(preds_all_base,true_all)
    return oof_scores


class StackingAveragedKerasModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=6):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y,tgt='Test',year = "year"):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=False, random_state=156)
        
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            k=0
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                
                        # Set up callbacks
                model_id = np.random.randint(low = 0,high = 10000,size = 1)
                fpath = f'./keras-ch/{k}_{tgt}_{i}_{year}.h5'
                callbacks = [EarlyStopping(monitor='val_loss', patience=20),TQDMNotebookCallback(leave_inner=False),#ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1),
                     ModelCheckpoint(filepath=fpath, monitor='val_loss', save_best_only=True)]
                fit_params = {"epochs":150,
                     "verbose":0,
                     "batch_size":64,
                     "callbacks":callbacks}
                X_valid = np.array(X)[holdout_index,:]
                y_valid = np.array(y)[holdout_index]
                
                X_train = np.array(X)[train_index,:]
                y_train = np.array(y)[train_index]
                
                fit_params["validation_data"] = (X_valid, y_valid)
                print(X_train.shape)
                print(y_train.shape)
                print(X_valid.shape)
                print(y_valid.shape)
                
                
                instance.fit(X_train, y_train,**fit_params)
                instance.model.load_weights(fpath)
                
                y_pred = instance.predict(np.array(X)[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                k=k+1
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

# Pull  KERAS models as basic models, do a stacking prediction

In [ ]:
import tensorflow as tf
def huber_loss(y_true, y_pred, clip_delta=1.0):
  error = y_true - y_pred
  cond  = tf.keras.backend.abs(error) < clip_delta

  squared_loss = 0.5 * tf.keras.backend.square(error)
  linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)

  return tf.where(cond, squared_loss, linear_loss)

def tilted_loss( y, f,q = 0.55):
    e = (y - f)
    return K.mean(K.maximum(q * e, (q - 1) * e),
                              axis=-1)

def huber_loss_mean(y_true, y_pred, clip_delta=1.0):
  return tf.keras.backend.mean(huber_loss(y_true, y_pred, clip_delta))


def create_a(optimizer=Adam(lr=0.01),
                 kernel_initializer=random_uniform(), 
                 dropout=0.1,input_shape = (1,175)):
    model = Sequential()
    model.add(Dense(1024,input_dim = input_shape,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(8,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1,kernel_initializer=kernel_initializer))
    model.compile(loss='mae',optimizer=optimizer, metrics=['mae'])

    return model
def create_b(optimizer=Adam(lr=0.01),
                 kernel_initializer=random_uniform(), 
                 dropout=0.1,input_shape = (1,175)):
    model = Sequential()
    model.add(Dense(1024,input_dim = input_shape,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(8,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1,kernel_initializer=kernel_initializer))
    model.compile(loss='mae',optimizer=optimizer, metrics=['mae'])

    return model
def create_c(optimizer=Adam(lr=0.01),
                 kernel_initializer=random_uniform(), 
                 dropout=0.1,input_shape = (1,175)):
    model = Sequential()
    model.add(Dense(96,input_dim = input_shape,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(16,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1,kernel_initializer=kernel_initializer))
    model.compile(loss='mae',optimizer=optimizer, metrics=['mae'])

    return model
def create_d(optimizer=Adam(lr=0.01),
                 kernel_initializer=random_uniform(), 
                 dropout=0.1,input_shape = (1,175)):
    model = Sequential()
    model.add(Dense(1024,input_dim = input_shape,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(8,kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1,kernel_initializer=kernel_initializer))
    model.compile(loss='mae',optimizer=optimizer, metrics=['mae'])

    return model



mod_a = KerasRegressor(build_fn=create_a,verbose=0)
mod_b = KerasRegressor(build_fn=create_b,verbose=0)
mod_c = KerasRegressor(build_fn=create_c,verbose=0)
mod_d = KerasRegressor(build_fn=create_d,verbose=0)


stacked_averaged_models = StackingAveragedKerasModels(base_models = (mod_a,mod_b,mod_c,mod_d),
                                                 meta_model = lasso)
#,lgb_a,lgb_b,lgb_c,lgb_d),
#ax,score,oof_scores  = rmsle_cv_gen_compare(stacked_averaged_models,plot=True,X_base=X,y=y,X_alt=X_filt)

In [ ]:
#print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


# Final predictions:



In [ ]:
from sklearn.ensemble import IsolationForest
irf = IsolationForest(verbose =1,contamination='auto',behaviour='new')

def clean_target(y,wupper = int(24*10*2.5),wmean =int(24*8/0.4),wlower = int(24*4*2.5),wstd = int(24*30*2.5),upper_std=False ):
    import altair as alt
    alt.data_transformers.enable('default', max_rows=None)

    tmp = pd.DataFrame(y)
    tmp.columns=['Tgt']
    tmp.reset_index(inplace=True)

    tmp['mean'] = tmp['Tgt'].rolling(window = wmean,center=True).median().fillna(method='bfill')
    tmp['std'] = tmp['Tgt'].rolling(window = wstd,center=True).std().fillna(method='bfill')
    #tmp['upper'] = (tmp['mean']+ 1.5 * tmp['std']).fillna(method='bfill').fillna(method='ffill')
    tmp['lower'] = (tmp['mean']- 1.7 * tmp['std']).fillna(method='bfill').fillna(method='ffill')
    #tmp['lower'] = tmp['Tgt'].rolling(window = wlower,center=True).quantile(0.04).fillna(method='bfill')
    tmp['upper'] = tmp['Tgt'].rolling(window = wupper).quantile(0.98).fillna(method='bfill')
    if upper_std:
        tmp['upper'] = (tmp['mean']+ 1.7 * tmp['std']).fillna(method='bfill').fillna(method='ffill')


    tmp_melt = tmp.melt(id_vars = ['date'])
    y = y.loc[tmp[(tmp.Tgt > tmp.lower) & (tmp.Tgt < tmp.upper)].date,]
    
    ch = alt.Chart(width=1000,data = tmp_melt).mark_point(filled=True,size= 5).encode(
                x='date',
                y='value:Q',
                color='variable:N',
                #shape = 'variable',
                opacity=alt.value(0.6))
    
    
    return y,tmp_melt,ch


preds_all = []
model = stacked_averaged_models

scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),StandardScaler(),PCA(whiten=True))

for year in [2016,2017]:
    print(year)
    year_train = 2019
    data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')
    X_test = data_dict[year]['X_test_ts'].copy().filter(regex  ="rougher",axis = 1)
    
    tgt = "rougher.output.recovery"
    
    X = data_dict[year_train]['X_train_tsclean'].copy().tz_convert(None)
    print(f'1a) Test shape: {X_test.shape}, train: {X.shape}')
    y = data_dict[year_train]['y_train_tsclean'][tgt].copy().dropna().tz_convert(None)
    y = y[(y>45) & (y <99)]
    #Fit Isolation RF
    #irf.fit(X)
    #irf_preds = irf.predict(X)
   # X= X.loc[irf_preds > 0]
    print(f'1b) Test shape: {X_test.shape}, train: {X.shape} after FILT')
    
    #y,tmp_melt,chrr = clean_target(y,upper_std=True)
   # print(ch)

    inds = X.index.intersection(y.index)
    X = X.loc[inds].filter(regex  ="rougher",axis = 1)
    y = y.loc[inds]
    print(f'1c) Test shape: {X_test.shape}, train: {X.shape} after intersect with Y')
    scaler.fit(X)
    Xsc = scaler.transform(X)
    
    mod_a = KerasRegressor(build_fn=create_a,input_shape = Xsc.shape[1],verbose=0)
    mod_b = KerasRegressor(build_fn=create_b,input_shape = Xsc.shape[1],verbose=0)
    mod_c = KerasRegressor(build_fn=create_c,input_shape = Xsc.shape[1],verbose=0)
    mod_d = KerasRegressor(build_fn=create_d,input_shape = Xsc.shape[1],verbose=0)
    model = StackingAveragedKerasModels(base_models = (mod_a,mod_b,mod_c,mod_d),
                                                     meta_model = lasso)
    
    model.fit(Xsc, y,tgt=tgt,year = year)
    Xtestsc = scaler.transform(X_test)
    
    ypred_r = model.predict(Xtestsc)
    preds_r = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_r}).set_index('date')
    
    tgt = "final.output.recovery"
    
    X = data_dict[year_train]['X_train_tsclean'].copy().tz_convert(None)
    X_test = data_dict[year]['X_test_ts'].copy()
    y = data_dict[year_train]['y_train_tsclean'][tgt].copy().dropna().tz_convert(None)
    y = y[(y>35) & (y <97)]
    print(f'2a) Test shape: {X_test.shape}, train: {X.shape}')
    # Data cleaning:
   
   # y,tmp_melt,chf = clean_target(y,wupper =24*10*5,wstd=24*50,upper_std=True)
   # print(ch)
   # irf.fit(X)
   # irf_preds = irf.predict(X)
   # X= X.loc[irf_preds > 0]
    print(f'2b) Test shape: {X_test.shape}, train: {X.shape} after FILT')
    
    inds = X.index.intersection(y.index)
    X = X.loc[inds]
    y = y.loc[inds]
    print(f'2c) Test shape: {X_test.shape}, train: {X.shape} after intersect with Y')
    scaler.fit(X)
    Xsc = scaler.transform(X)
    
    Xtestsc = scaler.transform(X_test)
    mod_a = KerasRegressor(build_fn=create_a,input_shape = Xsc.shape[1],verbose=0)
    mod_b = KerasRegressor(build_fn=create_b,input_shape = Xsc.shape[1],verbose=0)
    mod_c = KerasRegressor(build_fn=create_c,input_shape = Xsc.shape[1],verbose=0)
    mod_d = KerasRegressor(build_fn=create_d,input_shape = Xsc.shape[1],verbose=0)
    model = StackingAveragedKerasModels(base_models = (mod_a,mod_b,mod_c,mod_d),
                                                     meta_model = lasso)
    
    model.fit(Xsc, y,tgt=tgt,year = year)
    ypred_f = model.predict(Xtestsc)
    preds_f = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_f}).set_index('date')

    preds_all.append(preds_r.join(preds_f))


In [ ]:
stacked_preds_sub = pd.concat(preds_all)
stacked_preds_sub = stacked_preds_sub.reset_index()
stacked_preds_sub['date'] = stacked_preds_sub['date'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
stacked_preds_sub.set_index('date',inplace=True)
stacked_preds_sub.drop_duplicates(inplace=True)
stacked_preds_sub.to_csv('../results/stacked_sub_lgb_lasso_base_alldata_tsclean_keras.csv')

In [ ]:
stacked_preds_sub.plot(x='rougher.output.recovery',y='final.output.recovery',xlim=(60,100),ylim=(30,100),kind='scatter')

In [ ]:
ypred_r

# Average Keras Models

In [ ]:
# Preload the models according to the architecture
tgt = ['rougher.output.recovery','final.output.recovery']
year = [2016,2017]
models_final = {}
models_rougher = {}
# 332 , 175

list_models_rougher = [create_a(input_shape = 175),create_b(input_shape = 175),create_c(input_shape = 175),create_d(input_shape = 175)]
list_models_final = [create_a(input_shape = 332),create_b(input_shape = 332),create_c(input_shape = 332),create_d(input_shape = 332)]

for i in range(4):
    for fold in range(6):
        for y in year:
            models_rougher[f'{fold}-{y}-{i}'] = list_models_rougher[i]
            fpathr = f'./keras-ch/{fold}_{tgt[0]}_{i}_{y}.h5'
            models_rougher[f'{fold}-{y}-{i}'].model.load_weights(fpathr)
    
            #5_final.output.recovery_1_2017
            models_final[f'{fold}-{y}-{i}'] = list_models_final[i]
            fpathf = f'./keras-ch/{fold}_{tgt[1]}_{i}_{y}.h5'
            models_final[f'{fold}-{y}-{i}'].model.load_weights(fpathf)
    

In [ ]:

year_train = 2019
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')
X_test = data_dict[2019]['X_test_ts'].copy().filter(regex  ="rougher",axis = 1)
tgt = "rougher.output.recovery"
X = data_dict[year_train]['X_train_tsclean'].copy().tz_convert(None)
print(f'1a) Test shape: {X_test.shape}, train: {X.shape}')
y = data_dict[year_train]['y_train_tsclean'][tgt].copy().dropna().tz_convert(None)
y = y[(y>45) & (y <99)]
print(f'1b) Test shape: {X_test.shape}, train: {X.shape} after FILT')
inds = X.index.intersection(y.index)
X = X.loc[inds].filter(regex  ="rougher",axis = 1)
y = y.loc[inds]
scaler.fit(X)
Xsc = scaler.transform(X)
Xtestsc_r = scaler.transform(X_test)

# ypred_r = model.predict(Xtestsc)
# preds_r = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_r}).set_index('date')
    
tgt = "final.output.recovery"
X = data_dict[year_train]['X_train_tsclean'].copy().tz_convert(None)
X_test = data_dict[2019]['X_test_ts'].copy()
y = data_dict[year_train]['y_train_tsclean'][tgt].copy().dropna().tz_convert(None)
y = y[(y>35) & (y <97)]
print(f'2a) Test shape: {X_test.shape}, train: {X.shape}')
scaler.fit(X)
Xsc = scaler.transform(X)
Xtestsc_f = scaler.transform(X_test)

# ypred_f = model.predict(Xtestsc)
# preds_f = pd.DataFrame(data = {'date':X_test.index, tgt:ypred_f}).set_index('date')

# preds_all.append(preds_r.join(preds_f))

In [ ]:
pr_all = np.zeros((Xtestsc_f.shape[0],2))
k=0
for i in range(4):
    for fold in range(6):
        for y in year:
            k+=1
            pr_all[:,0]+= models_rougher[f'{fold}-{y}-{i}'].predict(Xtestsc_r).reshape((-1,))
            pr_all[:,1]+= models_final[f'{fold}-{y}-{i}'].predict(Xtestsc_f).reshape((-1,))
pr_all = pr_all / k
preds_all = pd.DataFrame(data = pr_all,columns = ['rougher.output.recovery','final.output.recovery'],index = X_test.index)

In [ ]:
preds_all.plot(figsize = (20,6),style = ['o','o'])